<a href="https://colab.research.google.com/github/YESICA287/Infraestructura-de-Big-Data/blob/main/Act10IBDTema10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Tema 10 Infraestructura de Big Data

<img src=https://aitor-medrano.github.io/bigdata2122/imagenes/spark/01spark-integration.png>

In [1]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=065d02b19ebe641a20c9a2202e6ff5a14492f13b0263973e476349b037955b47
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


#RDD

Un RDD (Resilient Distributed Datasets) es una estructura de datos que abstrae los datos para su procesamiento en paralelo.

Antes de Spark 2.0, los RDD eran el interfaz principal para interactuar con los datos.

Se trata de una colección de elementos tolerantes a fallos que son immutables (una vez creados, no se pueden modificar) y diseñados para su procesamiento distribuido. Cada conjunto de datos en los RDD se divide en particiones lógicas, que se pueden calcular en diferentes nodos del clúster.

Hay dos formas de crear un RDD:

**Paralelizando una colección ya existente en nuestra aplicación Spark.
Referenciando un dataset de un sistema externo como HDFS, HBase, etc...**

Sobre los RDD se pueden realizar dos tipos de operaciones:

**Acción: devuelven un valor tras ejecutar una computación sobre el conjunto de datos.
Transformación: es una operación perezosa que crea un nuevo conjunto de datos a partir de otro RDD/Dataset, tras realizar un filtrado, join, etc..**

###Creamos la sesión

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext


##Acciones

###Paralelizamos

In [3]:
miRDD = sc.parallelize([1,2,3,4,5,6,7,8,9])

lista  = ['Hola', 'Adiós', 'Hasta luego']
listaRDD = sc.parallelize(lista) # Creamos un RDD a partir de una lista
listaRDD4 = sc.parallelize(lista,4) # Creamos un RDD con 4 particiones

###Take Sample
Cuando queremos recuperar un número determinado de resultado, de forma similar a limit en SQL, tenemos la acción take:

In [4]:
miRDD.take(3)       # [1, 2, 3]
listaRDD.take(2)    # ['Hola', 'Adiós']

['Hola', 'Adiós']

Otra opción es utilizar sample para obtener una muestra de los datos, aunque en este caso no es una acción sino una transformación, ya que va a obtener información posiblemente de varias particiones, teniendo que barajar los datos (shuffle):

In [5]:
miRDDmuestra = miRDD.sample(False, 0.5)
miRDDmuestra.collect()  # [2, 4, 6, 7, 8, 9] / [1, 2, 3, 4, 6] / [5, 8, 9]

[2, 3, 4, 5, 9]

In [13]:
# Importar desde drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Transformaciones Narrow

###Map

La transformación map aplica la función recibida a cada elemento del RDD, de manera que vamos a poder añadir una nueva columna, modificar una existente, etc...

Por ejemplo, si la entrada es un RDD que contiene [1, 2, 3, 4, 5], al hacer rdd.map(x=>x*2) obtendríamos un nuevo RDD con [2, 4, 6, 8, 10]:

In [14]:
rdd = sc.textFile("/content/drive/MyDrive/Tecmilenio 5TO Y 6TO/Infraestructura Big Data/empleados.txt")
rdd.count()                 # 4 - cantidad de líneas
resultRDD = rdd.map(len)    # obtenemos la cantidad de caracteres cada línea
resultRDD.collect()         # [61, 52, 60, 50]

[61, 52, 60, 50]

In [15]:
empleados = rdd.collect()
for empleado in empleados:
    print(empleado)

Michael|Montreal,Toronto|Male,30|DB:80|Product:DeveloperLead
Will|Montreal|Male,35|Perl:85|Product:Lead,Test:Lead
Shelley|New York|Female,27|Python:80|Test:Lead,COE:Architect
Lucy|Vancouver|Female,57|Sales:89,HR:94|Sales:Lead


###FlatMap

La transformación flatMap es muy similar a la anterior, pero en vez de devolver una lista con un elemento por cada entrada, devuelve una única lista deshaciendo las colecciones en elementos individuales:

In [17]:
rdd = sc.textFile("/content/drive/MyDrive/Tecmilenio 5TO Y 6TO/Infraestructura Big Data/empleados.txt")
resultFM = rdd.flatMap(lambda x: x.split("|"))
resultFM.collect()

['Michael',
 'Montreal,Toronto',
 'Male,30',
 'DB:80',
 'Product:Developer\x04Lead',
 'Will',
 'Montreal',
 'Male,35',
 'Perl:85',
 'Product:Lead,Test:Lead',
 'Shelley',
 'New York',
 'Female,27',
 'Python:80',
 'Test:Lead,COE:Architect',
 'Lucy',
 'Vancouver',
 'Female,57',
 'Sales:89,HR:94',
 'Sales:Lead']

###Filter

Permite filtrar los elementos que cumplen una condición mediante filter:

In [18]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
resultRDD = rdd.filter(lambda x: x%2==0)
resultRDD.collect()     # [2, 4]

[2, 4]

También podemos anidar diferentes transformaciones. Para este ejemplo, vamos a crear tuplas formadas por un número y su cuadrado, luego quitar los que no coincide el número su potencia (el 0 y el 1), y luego aplanarlo en una lista:

In [19]:
rdd10 = sc.parallelize(range(10+1))
rddPares = rdd10.map(lambda x: (x, x**2)).filter(lambda x: (x[0] != x[1])).flatMap(lambda x: x)
rddPares.collect()      # [2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81, 10, 100]

[2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81, 10, 100]

Veamos otro ejemplo. Retomamos los datos de los empleados y si queremos filtrar los empleados que son hombres, primero separamos por las | y nos quedamos con el tercer elemento que contiene el sexo y la edad. A continuación, separamos por la coma para quedarnos en el sexo en la posición 0 y la edad en el 1, y comparamos con el valor deseado:

In [20]:
rdd = sc.textFile("/content/drive/MyDrive/Tecmilenio 5TO Y 6TO/Infraestructura Big Data/empleados.txt")
hombres = rdd.filter(lambda x: x.split("|")[2].split(",")[0] == "Male")
resultFM.collect()

['Michael',
 'Montreal,Toronto',
 'Male,30',
 'DB:80',
 'Product:Developer\x04Lead',
 'Will',
 'Montreal',
 'Male,35',
 'Perl:85',
 'Product:Lead,Test:Lead',
 'Shelley',
 'New York',
 'Female,27',
 'Python:80',
 'Test:Lead,COE:Architect',
 'Lucy',
 'Vancouver',
 'Female,57',
 'Sales:89,HR:94',
 'Sales:Lead']

###Distinct
Si utilizamos distinct eliminaremos los elementos repetidos:

In [21]:
rdd = sc.parallelize([1,1,2,2,3,4,5])
resultRDD = rdd.distinct()
resultRDD.collect()     # [4, 1, 5, 2, 3]

[2, 4, 1, 3, 5]

##Caso práctico

In [23]:
from google.colab import drive
drive.mount('/content/drive')
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

quijoteTxt = spark.read.text("/content/drive/MyDrive/Tecmilenio 5TO Y 6TO/Infraestructura Big Data/el_quijote.txt")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


¿Cúantas líneas de texto contiene el archivo el_quijote?

In [25]:
quijoteTxt.count()

12380

Crear un DataFrame con las líneas que contiene las palabras "Dulcinea del Toboso" e imprima cuantas línes son

In [29]:
df_dulcinea = quijoteTxt.filter(quijoteTxt.value.contains("Dulcinea del Toboso"))
print("Número de líneas con la palabra 'Dulcinea del Toboso':", df_dulcinea.count())

Número de líneas con la palabra 'Dulcinea del Toboso': 12


Crear un DataFrame con las líneas que contiene la palabra "Sancho" e imprima cuantas línes son

In [28]:
df_sancho = quijoteTxt.filter(quijoteTxt.value.contains("Sancho"))
print("Número de líneas con la palabra 'Sancho':", df_sancho.count())

Número de líneas con la palabra 'Sancho': 775


Crear un DataFrame con las líneas que contiene las palabra "molino" e imprima cuantas línes son

In [30]:
df_molino = quijoteTxt.filter(quijoteTxt.value.contains("molino"))
print("Número de líneas con la palabra 'molino':", df_molino.count())

Número de líneas con la palabra 'molino': 15


Crear un Map de cada elemento RDD con la finalidad de imprimir cada línea del texto quijote

In [33]:
lines = quijoteTxt.rdd.map(lambda line: line.value)
lines.collect()

['The Project Gutenberg eBook of El Quijote apócrifo',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States,',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '',
 'Title: El Quijote apócrifo',
 '',
 '',
 'Author: Alonso Fernández de Avellaneda',
 '',
 'Release date: December 9, 2019 [eBook #60884]',
 '',
 'Language: Spanish',
 '',
 'Credits: Ramon Pajares Box and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images generously made available by Biblioteca Digital Hispánica/Biblioteca Nacional de España.)',
 '',
 '',
 '*** START OF THE PROJECT GUTENBERG EBOOK EL QUIJOTE 

Crear un FlatMap para separar el texto por commas "," e imprima su resultado

In [32]:
quijoteTxt.rdd.flatMap(lambda line: line.value.split(",")).collect()

['The Project Gutenberg eBook of El Quijote apócrifo',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it',
 ' give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States',
 '',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '',
 'Title: El Quijote apócrifo',
 '',
 '',
 'Author: Alonso Fernández de Avellaneda',
 '',
 'Release date: December 9',
 ' 2019 [eBook #60884]',
 '',
 'Language: Spanish',
 '',
 'Credits: Ramon Pajares Box and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images generously made available by Biblioteca Digital Hispánica/Biblioteca Nacional de España.)',
 '',
 '',
 '*** START OF THE PROJECT GUTENBERG EBOOK